In [ ]:
import os
import re
import time
import json
import requests
import logging as log
from dotenv import load_dotenv

load_dotenv()


In [ ]:
goal = "I need few developers for my new startup, I am looking for people with 2 years of experience in python and react."
location = "Bengaluru, Karnataka, India"

In [ ]:
urls = [
    "https://keralatourbus.com/",
    "https://www.justdial.com/Thrissur/Bus-On-Rent-For-Wedding-in-Kerala/nct-11275471",
    "https://www.sulekha.com/bus-rentals/trivandrum",
    "https://www.shaadibaraati.com/wedding-transportation/kerala/oneness-travels/MGyvjmIxtV",
    "https://devannmpd.wixsite.com/taxicarkerala/tourist-buses-in-cochin",
    "https://www.asparkholidays.com/cochin/luxury-bus-hire",
    "https://www.redbus.in/bus-hire/wedding",
    "https://www.asparkholidays.com/thiruvananthapuram/book-volvo-bus",
    "https://www.justdial.com/Ernakulam/Bus-On-Rent-For-Wedding/nct-11275471",
]

In [ ]:
def remove_html_junk(data):
    # remove all class names
    data = re.sub(r'(?is)class="[^"]*"', '', data)
    data = re.sub(r'(?is)className="[^"]*"', '', data)

    # remove all inline styles
    data = re.sub(r'(?is)style="[^"]*"', '', data)

    # remove all aria-label
    data = re.sub(r'(?is)aria-label="[^"]*"', '', data)

    # remove all comments
    data = re.sub(r'(?s)<!--(.*?)-->[\n]?', '', data)

    # remove all ids
    data = re.sub(r'(?is)id="[^"]*"', '', data)

    # remove all newlines, long space and tabs
    data = re.sub(r'[\n\t]', ' ', data)
    data = re.sub(r' +', ' ', data)

    # remove all non-imp tags both open and close
    remove_tags = ["span", "div", "strong", "h1", "h2", "h3", "h4", "h5", "h6", "b", "li", "p", "ui", "section"]   
    for tag in remove_tags:
        data = re.sub(r'(?is)<{}[^>]*>'.format(tag), '', data)
        data = re.sub(r'(?is)</{}[^>]*>'.format(tag), '', data) 

    return data

import requests
from bs4 import BeautifulSoup
import html2text

h = html2text.HTML2Text()

def scrape_data(url):
    log.info(f"\nScraping data from {url}")
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        
        unwanted_tags = [
            "pre",
            "code",
            "blockquote",
            "em",
            "br",
            "source",
            "circle",
            "svg",
            "img",
            "button",
            "input",
            "form",
            "footer",
            "header",
            "aside",
            "nav",
            "script",
            "style",
            "noscript",
            "iframe",
            "meta",
            "head",
            
        ]
        for tag in unwanted_tags:
            for match in soup.find_all(tag):
                match.decompose()
        if soup.body is None:
            log.warning(f"Error: Unable to fetch data from {url}")
            return ""
        try:
            # data = soup.body.prettify()
            data = soup.main.prettify()
        except:
            data = soup.body.prettify()

        data = remove_html_junk(data)

    
        log.info(f"Successfully scraped data from {url}")   
        return data

    else:
        log.warning(
            f"Error: Unable to fetch data from {url}. Status code: {response.status_code}"
        )
        return ""
